In [22]:
import tensorflow as tf
import math
# Import the image
image = tf.image.decode_jpeg(tf.read_file("./images/DSC_0059.jpg"), channels=3)


session = tf.InteractiveSession()

print(session.run(image))




[[[32 36 39]
  [38 42 45]
  [33 37 40]
  ...
  [44 44 46]
  [40 40 42]
  [44 44 46]]

 [[37 41 44]
  [36 40 43]
  [34 38 41]
  ...
  [34 34 36]
  [32 32 34]
  [40 40 42]]

 [[42 46 49]
  [38 42 45]
  [37 41 44]
  ...
  [41 41 43]
  [36 36 38]
  [39 39 41]]

 ...

 [[18 18 18]
  [14 14 14]
  [14 14 14]
  ...
  [24 24 26]
  [22 22 22]
  [18 18 18]]

 [[16 16 16]
  [13 13 13]
  [13 13 13]
  ...
  [24 24 26]
  [22 22 22]
  [18 18 18]]

 [[14 14 14]
  [13 13 13]
  [13 13 13]
  ...
  [24 24 26]
  [22 22 22]
  [18 18 18]]]


In [31]:
img_shape = [4795, 3197]
img_h = 4795
img_w = 3197
height = img_h // 10
width = img_w // 10
vertical_margin = img_h // 100
horizontal_margin = img_w // 100
lim_top = img_h - img_h // vertical_margin - height  
lim_left = img_w - img_w // horizontal_margin - width

t = tf.random_uniform(
        [], minval=vertical_margin, maxval=lim_top, dtype=tf.int32)
l = tf.random_uniform(
        [], minval=horizontal_margin, maxval=lim_left, dtype=tf.int32)
h = tf.constant(height)
w = tf.constant(width)

bbox = (t, l, h, w)
print(bbox)

47
(<tf.Tensor 'random_uniform_5:0' shape=() dtype=int32>, <tf.Tensor 'random_uniform_6:0' shape=() dtype=int32>, <tf.Tensor 'Const_18:0' shape=() dtype=int32>, <tf.Tensor 'Const_19:0' shape=() dtype=int32>)


In [24]:
output_image = tf.image.encode_jpeg(masked_image)

# Create a constant as filename
file_name = tf.constant('Output_image.jpeg')
file = tf.write_file(file_name, output_image)

#print(session.run(file))

print("Image Saved!")

session.close()

Image Saved!
